In [22]:
import pandas as pd
import sqlite3

## 1. Create a connection to the database using the sqlite3 library.

In [23]:
db_path = "../data/checking-logs.sqlite"
conn = sqlite3.connect(db_path)
conn.execute("DROP TABLE IF EXISTS datamart;")

## 2. Create a new table called datamart in the database by joining the tables pageviews and checker using only one query.
- The table should have the following columns: "uid", "labname", "first_commit_ts", and "first_view_ts".
- "first_commit_ts" is a new name for the "timestamp" column in the checker table. It shows the first commit from a particular lab and user.
- "first_view_ts" shows the first time a user visited the pageviews table. It is the timestamp of when a user visited the newsfeed.
- status = 'ready' should still be a filter.
- numTrials = 1 should still be a filter.
- "labnames" should be from the list: laba04, laba04s, laba05, laba06, laba06s, and project1.
- The table should contain only users (uids with user_*), not admins.
- "first_commit_ts" and "first_view_ts" should be parsed as datetime64[ns].

In [24]:
query = """
CREATE TABLE datamart AS
SELECT 
    c.uid,
    c.labname,
    MIN(c.timestamp) AS first_commit_ts,
    MIN(p.datetime) AS first_view_ts
FROM 
    checker c
LEFT JOIN 
    pageviews p ON c.uid = p.uid
WHERE 
    c.status = 'ready' 
    AND c.numTrials = 1
    AND c.labname IN ('laba04', 'laba04s', 'laba05', 'laba06', 'laba06s', 'project1')
    AND c.uid LIKE 'user_%'
GROUP BY 
    c.uid, c.labname;
"""
conn.execute(query)

## 3. Using Pandas methods, create two dataframes: test and control.
- test should contain users with values in "first_view_ts".

In [25]:
datamart = pd.read_sql_query("SELECT * FROM datamart", conn)
datamart['first_commit_ts'] = pd.to_datetime(datamart['first_commit_ts'])
datamart['first_view_ts'] = pd.to_datetime(datamart['first_view_ts'])

- control should contain users with missing values in "first_view_ts".


In [26]:
test = datamart.dropna(subset=['first_view_ts']).copy()
control = datamart[datamart['first_view_ts'].isna()].copy()

- Replace the missing values in the control table with the average "first_view_ts" value of the test users. We will use this value for future analyses.

In [27]:
if not test.empty:
    avg_first_view_ts = test['first_view_ts'].mean()
    control['first_view_ts'] = avg_first_view_ts

In [28]:
control.head()

,uid,labname,first_commit_ts,first_view_ts
12,user_11,laba05,2020-05-03 21:06:55.970293,2020-04-27 00:40:05.761783552
13,user_11,project1,2020-05-03 23:45:33.673409,2020-04-27 00:40:05.761783552
14,user_12,laba04,2020-04-18 17:07:51.767358,2020-04-27 00:40:05.761783552
15,user_12,laba04s,2020-04-26 15:42:38.070593,2020-04-27 00:40:05.761783552
16,user_12,laba05,2020-05-03 08:39:25.174316,2020-04-27 00:40:05.761783552


- Save both tables in the database. You will use them in the next exercises.

In [29]:
test.to_sql('test', conn, if_exists='replace', index=False)
control.to_sql('control', conn, if_exists='replace', index=False)

81

## 4. Close the connection.

In [30]:
test = pd.io.sql.read_sql("SELECT * FROM test;", conn)
test.head()

,uid,labname,first_commit_ts,first_view_ts
0,user_1,laba04,2020-04-26 17:06:18.462708,2020-04-26 21:53:59.624136
1,user_1,laba04s,2020-04-26 17:12:11.843671,2020-04-26 21:53:59.624136
2,user_1,laba05,2020-05-02 19:15:18.540185,2020-04-26 21:53:59.624136
3,user_1,laba06,2020-05-17 16:26:35.268534,2020-04-26 21:53:59.624136
4,user_1,laba06s,2020-05-20 12:23:37.289724,2020-04-26 21:53:59.624136


In [31]:
control = pd.io.sql.read_sql("SELECT * FROM control;", conn)
control.head()

,uid,labname,first_commit_ts,first_view_ts
0,user_11,laba05,2020-05-03 21:06:55.970293,2020-04-27 00:40:05.761783
1,user_11,project1,2020-05-03 23:45:33.673409,2020-04-27 00:40:05.761783
2,user_12,laba04,2020-04-18 17:07:51.767358,2020-04-27 00:40:05.761783
3,user_12,laba04s,2020-04-26 15:42:38.070593,2020-04-27 00:40:05.761783
4,user_12,laba05,2020-05-03 08:39:25.174316,2020-04-27 00:40:05.761783


In [32]:
conn.close()

In [33]:
control.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 81 entries, 0 to 80
Data columns (total 4 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   uid              81 non-null     object
 1   labname          81 non-null     object
 2   first_commit_ts  81 non-null     object
 3   first_view_ts    81 non-null     object
dtypes: object(4)
memory usage: 2.7+ KB


In [34]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 59 entries, 0 to 58
Data columns (total 4 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   uid              59 non-null     object
 1   labname          59 non-null     object
 2   first_commit_ts  59 non-null     object
 3   first_view_ts    59 non-null     object
dtypes: object(4)
memory usage: 2.0+ KB


In [35]:
datamart.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 140 entries, 0 to 139
Data columns (total 4 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   uid              140 non-null    object        
 1   labname          140 non-null    object        
 2   first_commit_ts  140 non-null    datetime64[ns]
 3   first_view_ts    59 non-null     datetime64[ns]
dtypes: datetime64[ns](2), object(2)
memory usage: 4.5+ KB
